In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from skimage import transform
import torchvision.transforms as transforms
import torchvision
from torch.autograd import Variable
import numpy as np;
from torch.utils.data import Dataset, DataLoader
import random;
import math;
import os
import tensorflow as tf
os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Computer Vision Competition

This computer vision competition is on something called "novelty detection", which is where the goal is being able to learn to detect novelties. In other words, you are given a training set of data, $D_{train}$, and at test time you are given $D_{test}$ which contains both data types from $D_{train}$ as well as novel data that has not been seen before. The task is that, at test time, your model will need to classify an image as a type from $D_{train}$, or something novel. This is a twist on the classification problems we have gone over in this class.

For this problem, we will be using the Fashion MNIST dataset as the training data. More information about this dataset can be found here: https://github.com/zalandoresearch/fashion-mnist 

At test time, we will run the model you have trained on a mystery dataset that is a combination of new Fashion MNIST data and also novel data from a different dataset, and your model will be ranked by how well it is able to distinguish Fashion data from novel data.

Your model can output something like a probability (hint: use Sigmoid activation function at the end) or threshold cutoff that allows you to distinguish that an image is novel, and this will be used to perform a binary classification of sorts (either novel, or not novel). **The important thing is that at the very end, your model outputs either a 0 if you think the image is of the same type as the training data, and a 1 if your model thinks it is novel**. The highest accuracy of novelty detection will win this competition. There is a lot of creativity allowed in this project, so we recommend you brainstorm some new ideas or even read up on some novelty detection literature if you are interested. All code submitted must be your own.

There is some skeleton code given, but feel free to change as much or as little as you would like, as long as you train using **only** the FashionMNIST dataset. Here are a couple of ideas on how you could go about this to get you started:

* Traditional image classification assigns a probability of each label to an image, perhaps do something along the lines of thresholding so that if the highest probability an image has of being a certain label is still below $x$, it is novel
* Checking how close/far a test image is to the training images, and thresholding
* Looking at how close certain features (for example, 2 layers into a 4 layer CNN) may be to the training data

At the end of training, be sure to save your model weights (code has been provided to do so). We will test your model, and the following is what you will need to submit (please include all of the code in the provided turnin.py file):

* Code that includes
    * Model architecture
    * Preprocessing of images (must only use a variation of the given transform function)
    * (Optional) postprocessing, instructions on this are in the last cell
* Model weights in the form of a 'pth' file
* Brief writeup giving a high-level overview of what you implemented

In [2]:
# feel free to change how much or little preprocessing is done, and at test 
# time the same preprocessing will be done on the test data
# transform = transforms.Compose([
# transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

transform = transforms.Compose([
transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)



In [3]:
# Sample model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1=nn.ELU()
        nn.init.xavier_uniform(self.cnn1.weight)
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2=nn.ELU()
        nn.init.xavier_uniform(self.cnn2.weight)
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
#         self.cnn3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=2)
#         self.relu3=nn.ELU()      
#         nn.init.xavier_uniform(self.cnn3.weight)
#         self.maxpool3=nn.MaxPool2d(kernel_size=2)
#         self.cnn4 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=5,stride=1,padding=2)
#         self.relu4=nn.ELU()
#         nn.init.xavier_uniform(self.cnn4.weight)
#         self.maxpool4=nn.MaxPool2d(kernel_size=2)
        self.fcl=nn.Linear(32*7*7,10)

    def forward(self, x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out=self.maxpool1(out)
        out=self.cnn2(out)
        out=self.relu2(out)
        out=self.maxpool2(out)
#         out=self.cnn3(out)
#         out=self.relu3(out)
#         out=self.maxpool3(out)
#         out=self.cnn4(out)
#         out=self.relu4(out)
#         out=self.maxpool4(out)`
        out=out.view(out.size(0),-1)
        out=self.fcl(out)
        batch = list(x.size())[0]
        return out


net = Net()
torch.save(model.state_dict(), PATH)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if sys.path[0] == '':


NameError: name 'model' is not defined

In [4]:
# Sample hyperparameters
epochs = 10

In [5]:
# Sample loss functions and optimizers
criterion=nn.CrossEntropyLoss()
learning_rate=0.015# BCELoss if output is sigmoid is recommended
optimizer=torch.optim.SGD(net.parameters(),lr=learning_rate)

In [6]:
# Run training
number = len(train_loader)
for epoch in range(epochs): 

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs = outputs.squeeze().float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i == (number - 10):
            loss = running_loss / (number - 10)
            print("Epoch: {0}, Loss: {1}".format(epoch + 1, loss))
            running_loss = 0.0

print('Finished Training')
directory = './saved_model/'
if not os.path.exists(directory):
    os.makedirs(directory)
torch.save(net.state_dict(), './saved_model/model.pth')

KeyboardInterrupt: 

To make sure your code is written in a way that is compatible with our testing framework, please make sure that the following code below runs. It also has the benefit of showing you how well your model does at classifying images from the training distribution correctly. It does **not** contain any of the novel images your model will eventually be tested on.

In [ ]:
# Check that code works

directory = './saved_model/'
if not os.path.exists(directory):
    os.makedirs(directory)
torch.save(net.state_dict(), './saved_model/model.pth')
net = Net()
net.load_state_dict(torch.load('./saved_model/model.pth'))
net.eval()
f= open("result.txt","w+")
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=2)

corrects = 0
total = 0
with torch.no_grad():
    for j, loader in enumerate([test_loader]):
        for i, data in enumerate(loader, 0):
            inputs, labels = data
            inputs=Variable(inputs)
            outputs=net(inputs)
            _,predicted=torch.max(outputs.data,1)
            sm = torch.nn.Softmax() 
            probabilities = sm(outputs)
            thresh = 0.7
            for k in probabilities:
                k = torch.max(k)
                total += 1
                if k >= thresh:
                    k = 0
                    print(k)
                    f.write(str(k))
                    if i < sub_lim:
                        corrects += 1
                else:
                    k = 1
                    print(k)
                    f.write(str(k))
                    if i > sub_lim:
                        corrects += 1
            #Converted to probabilities
            # get the inputs
            # If needed, can add a line here that thresholds, or does something with the
            # 'outputs' variable to get it into a compatible format. An example is something
            # like _, outputs = torch.max(outputs.data, 1). If this line is needed, please
            # this along with your model architecture code labeled as 'postprocess'
#             corrects[j] += (predicted==labels).sum()
#             totals[j] += (labels.size(0))
            
f.close() 
accs = corrects / total
print("Accuracy for test in distribution data:", accs)
